In [1]:
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import s3fs
import scipy
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.data.aggregators as ag
import entropy.data.cleaners as cl
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.data.validators as vl
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

In [2]:
fs.ls("3di-data-mdb/raw/pieces")

['3di-data-mdb/raw/pieces/',
 '3di-data-mdb/raw/pieces/data_XX0.rds',
 '3di-data-mdb/raw/pieces/data_XX1.rds',
 '3di-data-mdb/raw/pieces/data_XX2.rds',
 '3di-data-mdb/raw/pieces/data_XX3.rds',
 '3di-data-mdb/raw/pieces/data_XX4.rds',
 '3di-data-mdb/raw/pieces/data_XX5.rds',
 '3di-data-mdb/raw/pieces/data_XX6.rds',
 '3di-data-mdb/raw/pieces/data_XX7.rds',
 '3di-data-mdb/raw/pieces/data_XX8.rds',
 '3di-data-mdb/raw/pieces/data_XX9.rds']

In [3]:
df = hd.read_txn_data("X77")

Time for read_txn_data                 : 50.51 seconds
